In [97]:
import requests
import json
import pandas as pd

In [106]:
import requests

url = (
    "https://content.toto.nl/content-service/api/v1/q/event-list?"
    "startTimeFrom=2024-11-10T23:00:00Z&"
    "startTimeTo=2024-11-21T22:59:59Z&"
    "liveNow=false&"
    "maxEvents=150&"
    "popularityLimit=5&"
    "orderEventsBy=popularity&"
    "maxMarkets=10&"
    "orderMarketsBy=displayOrder&"
    "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
    "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
    "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
    "eventSortsIncluded=MTCH&"
    "includeChildMarkets=true&"
    "prioritisePrimaryMarkets=true&"
    "includeCommentary=true&"
    "includeMedia=true&"
    "drilldownTagIds=11&"  # Soccer drilldown tag
    "categoryCode=soccer&"  # Soccer category code
    "lang=nl-NL&"
    "channel=I"
)

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
}

response = requests.get(url, headers=headers)

# Parsing the response
data = response.json()

# Display the JSON structure for verification
print(data)

{'data': {'events': [{'id': '5120927', 'externalIds': [{'id': '10954279', 'provider': 'betgenius'}, {'id': 'sr:match:46657169', 'provider': 'betradar'}], 'extKey': None, 'retailCode': None, 'name': 'Deportivo Riestra Afbc vs Vélez Sarsfield', 'active': True, 'displayed': True, 'status': 'ACTIVE', 'displayOrder': -65, 'popularityOrder': 0, 'channels': ['P', 'A', 'R', 'T', 'X', 'I', 'M', 'O'], 'sortCode': 'MTCH', 'startTime': '2024-11-11T19:00:00Z', 'started': False, 'liveNow': False, 'liveBettingAvailable': True, 'resulted': False, 'settled': False, 'cashoutAvailable': True, 'sportId': '17', 'competitionDrilldownTagId': '2736', 'raceNumber': 0, 'venue': None, 'isVoid': False, 'blurb': None, 'fixedOddsAvailable': True, 'rcpAvailable': True, 'statisticsAvailable': False, 'neutralVenue': False, 'meeting': None, 'teams': [{'id': None, 'side': 'HOME', 'name': 'Deportivo Riestra Afbc', 'alternativeNames': [{'type': 'FULL', 'value': 'Deportivo Riestra Afbc'}], 'code': None, 'externalId': None,

In [107]:
# Initialize an empty list to collect match data
matches = []

# Extract match details from each event
for event in data['data']['events']:
    match = {
        "event_id": event.get("id"),
        "match_name": event.get("name"),
        "start_time": event.get("startTime"),
        "home_team": next((team['name'] for team in event['teams'] if team['side'] == "HOME"), None),
        "away_team": next((team['name'] for team in event['teams'] if team['side'] == "AWAY"), None),
        "competition": event['type']['name'],
        "country": event['class']['name'],
        "sport": event['category']['name'],
    }
    
    # Extract odds, if available
    outcomes = event.get("markets", [{}])[0].get("outcomes", [])
    if outcomes:
        match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "H"), None)
        match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "A"), None)
    
    matches.append(match)

# Convert the match list to a DataFrame
matches_df = pd.DataFrame(matches)

# Display the resulting DataFrame
matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
0,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,2024-11-11T19:00:00Z,Deportivo Riestra Afbc,Vélez Sarsfield,Argentinië Superliga,Argentinië,Voetbal,5.50,1.63
1,5699848,Frankrijk vs Israël,2024-11-14T19:45:00Z,Frankrijk,Israël,UEFA Nations League,Europees,Voetbal,1.14,19.00
2,5699853,Kazachstan vs Oostenrijk,2024-11-14T15:00:00Z,Kazachstan,Oostenrijk,UEFA Nations League,Europees,Voetbal,10.00,1.28
3,5699864,Nederland vs Hongarije,2024-11-16T19:45:00Z,Nederland,Hongarije,UEFA Nations League,Europees,Voetbal,1.40,7.25
4,5699842,Portugal vs Polen,2024-11-15T19:45:00Z,Portugal,Polen,UEFA Nations League,Europees,Voetbal,1.33,8.75
...,...,...,...,...,...,...,...,...,...,...
145,5946617,Darlington FC vs Peterborough Sports FC,2024-11-12T19:45:00Z,Darlington FC,Peterborough Sports FC,Engeland National League Noord,Engeland,Voetbal,1.87,3.50
146,5849372,Defensor Sporting vs CA Progreso,2024-11-14T00:30:00Z,Defensor Sporting,CA Progreso,Uruguay Primera División,Uruguay,Voetbal,1.69,4.50
147,5849377,Deportivo Maldonado vs Racing Club Montevideo,2024-11-12T22:00:00Z,Deportivo Maldonado,Racing Club Montevideo,Uruguay Primera División,Uruguay,Voetbal,3.00,2.25
148,5939314,Doncaster Rovers vs Salford City FC,2024-11-16T15:00:00Z,Doncaster Rovers,Salford City FC,Engeland League 2,Engeland,Voetbal,1.67,4.50


In [ ]:
url = "https://content.toto.nl/content-service/api/v1/q/popular-bets-event-list?minPrice=1.25&maxPrice=9.90&combinable=true&numOutcomes=3&startTimeFrom=2024-11-09T13%3A32%3A35Z&startTimeTo=2024-12-10T13%3A32%3A35Z&marketSortsIncluded=HH%2CMR&lang=nl-NL&channel=I"

payload = {}
headers = {
  'accept': 'application/json',
  'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
}

response = requests.request("GET", url, headers=headers, data=payload)

match_data = json.loads(response.text)


In [92]:
# Initialize an empty list to collect match data
matches = []

# Extract match details from each event
for event in match_data['data']['events']:
    match = {
        "event_id": event.get("id"),
        "match_name": event.get("name"),
        "start_time": event.get("startTime"),
        "home_team": next((team['name'] for team in event['teams'] if team['side'] == "HOME"), None),
        "away_team": next((team['name'] for team in event['teams'] if team['side'] == "AWAY"), None),
        "competition": event['type']['name'],
        "country": event['class']['name'],
        "sport": event['category']['name'],
    }
    
    # Extract odds, if available
    outcomes = event.get("markets", [{}])[0].get("outcomes", [])
    if outcomes:
        match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "H"), None)
        match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "A"), None)
    
    matches.append(match)

# Convert the match list to a DataFrame
matches_df = pd.DataFrame(matches)

# Display the resulting DataFrame
matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
0,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,2024-11-11T19:00:00Z,Deportivo Riestra Afbc,Vélez Sarsfield,Argentinië Superliga,Argentinië,Voetbal,None,1.63
1,5786235,Como 1907 vs Fiorentina,2024-11-24T14:00:00Z,Como 1907,Fiorentina,Italië Serie A,Italië,Voetbal,None,2.34
2,5699853,Kazachstan vs Oostenrijk,2024-11-14T15:00:00Z,Kazachstan,Oostenrijk,UEFA Nations League,Europees,Voetbal,None,1.28


In [110]:
# Get all offers for all matches of Football event_ids
# Initialize an empty list to accumulate all data
all_data = []

# Loop over all event_ids in your matches_df
for event_id in matches_df['event_id']:
    url = f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?eventIds={event_id}&includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"

    payload = {}
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

    response = requests.get(url, headers=headers, data=payload)
    json_data = json.loads(response.text)

    # Initialize empty lists to store the data for this event
    event_ids = []
    event_names = []
    market_names = []
    outcome_names = []
    odds_decimal = []
    prices_numerator = []
    prices_denominator = []
    outcome_types = []
    outcome_subtypes = []

    # Iterate over the events, markets, and outcomes
    for event in json_data['data']['events']:  # Iterate over each event
        event_id = event['id']
        event_name = event['name']
        
        for market in event['markets']:  # Iterate over each market in the event
            market_name = market['name']
            
            for outcome in market['outcomes']:  # Iterate over outcomes for each market
                outcome_name = outcome['name']
                odds_decimal_value = outcome['prices'][0]['decimal']
                numerator = outcome['prices'][0]['numerator']
                denominator = outcome['prices'][0]['denominator']
                outcome_type = outcome['type']
                outcome_subtype = outcome['subType']
                
                # Append captured values to the respective lists
                event_ids.append(event_id)
                event_names.append(event_name)
                market_names.append(market_name)
                outcome_names.append(outcome_name)
                odds_decimal.append(odds_decimal_value)
                prices_numerator.append(numerator)
                prices_denominator.append(denominator)
                outcome_types.append(outcome_type)
                outcome_subtypes.append(outcome_subtype)

    # Create a temporary DataFrame for this batch
    temp_df = pd.DataFrame({
        'Event ID': event_ids,
        'Event Name': event_names,
        'Market Name': market_names,
        'Outcome Name': outcome_names,
        'Odds (Decimal)': odds_decimal,
        'Price Numerator': prices_numerator,
        'Price Denominator': prices_denominator,
        'Outcome Type': outcome_types,
        'Outcome SubType': outcome_subtypes
    })
    
    # Append the temporary DataFrame to the all_data list
    all_data.append(temp_df)

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

      Event ID                                 Event Name  \
0      5120927  Deportivo Riestra Afbc vs Vélez Sarsfield   
1      5120927  Deportivo Riestra Afbc vs Vélez Sarsfield   
2      5120927  Deportivo Riestra Afbc vs Vélez Sarsfield   
3      5120927  Deportivo Riestra Afbc vs Vélez Sarsfield   
4      5120927  Deportivo Riestra Afbc vs Vélez Sarsfield   
...        ...                                        ...   
14613  5939559         Dumbarton vs Queen Of The South FC   
14614  5939559         Dumbarton vs Queen Of The South FC   
14615  5939559         Dumbarton vs Queen Of The South FC   
14616  5939559         Dumbarton vs Queen Of The South FC   
14617  5939559         Dumbarton vs Queen Of The South FC   

                 Market Name                                   Outcome Name  \
0                  Resultaat                                     Gelijkspel   
1                  Resultaat                         Deportivo Riestra Afbc   
2                  Resultaat  

In [111]:
final_df

,Event ID,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,Resultaat,Gelijkspel,3.65,53,20,MR,D
1,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,Resultaat,Deportivo Riestra Afbc,5.50,9,2,MR,H
2,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,Resultaat,Vélez Sarsfield,1.63,63,100,MR,A
3,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,Dubbele Kans,Deportivo Riestra Afbc of Vélez Sarsfield,1.26,13,50,DC,3
4,5120927,Deportivo Riestra Afbc vs Vélez Sarsfield,Dubbele Kans,Deportivo Riestra Afbc of Gelijkspel,2.10,11,10,DC,1
...,...,...,...,...,...,...,...,...,...
14613,5939559,Dumbarton vs Queen Of The South FC,Half-Time / Full-Time,Gelijkspel / Queen Of The South FC,6.75,23,4,HF,6
14614,5939559,Dumbarton vs Queen Of The South FC,Half-Time / Full-Time,Gelijkspel / Dumbarton,5.50,9,2,HF,4
14615,5939559,Dumbarton vs Queen Of The South FC,Half-Time / Full-Time,Queen Of The South FC / Queen Of The South FC,4.75,15,4,HF,9
14616,5939559,Dumbarton vs Queen Of The South FC,Half-Time / Full-Time,Dumbarton / Gelijkspel,15.00,14,1,HF,2


In [69]:
# Get all bets for 1 match

# import pandas as pd

# # Initialize empty lists to store the data
# event_ids = []
# event_names = []
# market_names = []
# outcome_names = []
# odds_decimal = []
# prices_numerator = []
# prices_denominator = []
# outcome_types = []
# outcome_subtypes = []

# # Iterate over the events, markets, and outcomes
# for event in json_data['data']['events']:  # Iterate over each event
#     event_id = event['id']
#     event_name = event['name']
    
#     for market in event['markets']:  # Iterate over each market in the event
#         market_name = market['name']
        
#         for outcome in market['outcomes']:  # Iterate over outcomes for each market
#             outcome_name = outcome['name']
#             odds_decimal_value = outcome['prices'][0]['decimal']
#             numerator = outcome['prices'][0]['numerator']
#             denominator = outcome['prices'][0]['denominator']
#             outcome_type = outcome['type']
#             outcome_subtype = outcome['subType']
            
#             # Append captured values to the respective lists
#             event_ids.append(event_id)
#             event_names.append(event_name)
#             market_names.append(market_name)
#             outcome_names.append(outcome_name)
#             odds_decimal.append(odds_decimal_value)
#             prices_numerator.append(numerator)
#             prices_denominator.append(denominator)
#             outcome_types.append(outcome_type)
#             outcome_subtypes.append(outcome_subtype)

# # Create a DataFrame with the collected data
# df = pd.DataFrame({
#     'Event ID': event_ids,
#     'Event Name': event_names,
#     'Market Name': market_names,
#     'Outcome Name': outcome_names,
#     'Odds (Decimal)': odds_decimal,
#     'Price Numerator': prices_numerator,
#     'Price Denominator': prices_denominator,
#     'Outcome Type': outcome_types,
#     'Outcome SubType': outcome_subtypes
# })

# # Show the DataFrame
# df

    Event ID         Event Name     Market Name  \
0    5537946  FC Twente vs Ajax     Draw No Bet   
1    5537946  FC Twente vs Ajax     Draw No Bet   
2    5537946  FC Twente vs Ajax    Dubbele Kans   
3    5537946  FC Twente vs Ajax    Dubbele Kans   
4    5537946  FC Twente vs Ajax    Dubbele Kans   
..       ...                ...             ...   
451  5537946  FC Twente vs Ajax  Match Specials   
452  5537946  FC Twente vs Ajax  Match Specials   
453  5537946  FC Twente vs Ajax  Match Specials   
454  5537946  FC Twente vs Ajax  Match Specials   
455  5537946  FC Twente vs Ajax  Match Specials   

                            Outcome Name  Odds (Decimal)  Price Numerator  \
0                                   Ajax            1.75                3   
1                              FC Twente            2.08               27   
2                     Gelijkspel of Ajax            1.42               21   
3                      FC Twente of Ajax            1.33               33   
4 